In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.regularizers import l2

from preprocess import water_process
# from model.cnn import cnn_4layer, mlp_layer
from model.mlp import mlp_layer

2024-01-30 19:50:55.869738: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

### Preprocess

In [3]:
# ====== preprocessing parameters ======
n_hours = 72
k = 24 
masked_value = 1e-10
split_1 = 0.7
split_2 = 0.85

In [4]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = water_process(n_hours, k, masked_value, split_1, split_2)

Index(['MEAN_RAIN', 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2',
       'GATE_S26_1', 'GATE_S26_2', 'PUMP_S25B', 'PUMP_S26', 'HWS_S25A',
       'HWS_S25B', 'HWS_S26', 'WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26'],
      dtype='object')


In [5]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(67435, 96, 16) (14451, 96, 16) (14451, 96, 16) (67435, 96) (14451, 96) (14451, 96)


### Threshold

#### Precipitation 95 percentile

In [6]:
# train_X_mask_rain_acc = np.sum(train_X_mask[:, -24:, 0], axis=1)
# print(train_X_mask_rain_acc.shape)

# test_X_mask_rain_acc = np.sum(test_X_mask[:, -24:, 0], axis=1)
# print(test_X_mask_rain_acc.shape)

train_X_mask_rain_acc = train_X_mask[:, 0, 0]
print(train_X_mask_rain_acc.shape)

val_X_mask_rain_acc = val_X_mask[:, 0, 0]
print(val_X_mask_rain_acc.shape)

test_X_mask_rain_acc = test_X_mask[:, 0, 0]
print(test_X_mask_rain_acc.shape)

(67435,)
(14451,)
(14451,)


In [7]:
percentile = 95

merged_array = np.concatenate((train_X_mask_rain_acc, val_X_mask_rain_acc, test_X_mask_rain_acc))

percentile_rain = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_rain)

95th Percentile of Daily Rain: 0.010711610503493287


### train_X_mask_filter

In [8]:
train_X_mask_extreme = train_X_mask[train_X_mask_rain_acc > percentile_rain]
print(train_X_mask_extreme.shape)

train_ws_y_extreme = train_ws_y[train_X_mask_rain_acc > percentile_rain]
print(train_ws_y_extreme.shape)

(3251, 96, 16)
(3251, 96)


In [9]:
train_X_mask_normal = train_X_mask[train_X_mask_rain_acc <= percentile_rain]
print(train_X_mask_normal.shape)

train_ws_y_normal = train_ws_y[train_X_mask_rain_acc <= percentile_rain]
print(train_ws_y_normal.shape)

(64184, 96, 16)
(64184, 96)


### val_X_mask_filter

In [10]:
val_X_mask_extreme = val_X_mask[val_X_mask_rain_acc > percentile_rain]
print(val_X_mask_extreme.shape)

val_ws_y_extreme = val_ws_y[val_X_mask_rain_acc > percentile_rain]
print(val_ws_y_extreme.shape)

(536, 96, 16)
(536, 96)


In [11]:
val_X_mask_normal = val_X_mask[val_X_mask_rain_acc <= percentile_rain]
print(val_X_mask_normal.shape)

val_ws_y_normal = val_ws_y[val_X_mask_rain_acc <= percentile_rain]
print(val_ws_y_normal.shape)

(13915, 96, 16)
(13915, 96)


### test_X_mask_filter

In [12]:
test_X_mask_extreme = test_X_mask[test_X_mask_rain_acc > percentile_rain]
print(test_X_mask_extreme.shape)

test_ws_y_extreme = test_ws_y[test_X_mask_rain_acc > percentile_rain]
print(test_ws_y_extreme.shape)

(1030, 96, 16)
(1030, 96)


In [13]:
test_X_mask_normal = test_X_mask[test_X_mask_rain_acc <= percentile_rain]
print(test_X_mask_normal.shape)

test_ws_y_normal = test_ws_y[test_X_mask_rain_acc <= percentile_rain]
print(test_ws_y_normal.shape)

(13421, 96, 16)
(13421, 96)


### Model

In [41]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0  # 0.1
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 100
EPOCHS = 1000
BATCH = 512
opt_num = k*4
input_shape = train_X_mask.shape[1:]

In [50]:
model = mlp_layer(input_shape=input_shape,
                   mlp_unit1=mlp_unit1,
                   mlp_unit2=mlp_unit2,
                   mlp_unit3=mlp_unit3,
                   mlp_unit4=mlp_unit4,
                   mlp_unit5=mlp_unit5,
                   mlp_unit6=mlp_unit6,
                   mlp_unit7=mlp_unit7,
                   mlp_unit8=mlp_unit8,
                   dropout=dropout,
                   masked_value=-1,
                   opt_num=opt_num
                  )
# model.summary()

In [51]:
trained_cnn = load_model('../../saved_models/water_all_95_cp.h5')
# trained_cnn = load_model('../../saved_models2/water_all_weighted_IPF_95.h5')
# trained_cnn = load_model('../../saved_models2/water_all_weighted_EVT_95.h5') 
# trained_cnn = load_model('../../saved_models2/water_all_weighted_META_95.h5') 

freeze_layer_index = 7

for i, layer in enumerate(trained_cnn.layers):
    #print(i, layer)
    if i <= freeze_layer_index:
        layer.trainable = False


l2_strength = 1e-8
for i in range(freeze_layer_index+1, len(trained_cnn.layers)):
    trained_cnn.layers[i].kernel_regularizer = l2(l2_strength)


trained_cnn._name = 'all_95_ft'
# trained_cnn.summary()

In [53]:
trained_cnn.compile(optimizer='adam',
                    loss='mse',
#                     loss=penalty_loss,
                    metrics=['mae']
                   )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint(
#                       '../../saved_models/water_all_weighted_META_95_ft.h5', 
#                      '../../saved_models/water_all_weighted_EVT_95_ft.h5', 
#                      '../../saved_models/water_all_weighted_IPF_95_ft.h5', 
                     '../../saved_models/water_all_95_ft.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True,
                    )


trained_cnn.fit(train_test_X, train_test_y,
                validation_data=(val_X_mask_extreme, val_ws_y_extreme),
                epochs=EPOCHS,
                batch_size=BATCH,
                verbose=2,
                shuffle=True,
                callbacks=[es, mc]
               )

Epoch 1/1000
7/7 - 3s - loss: 0.0162 - mae: 0.0988 - val_loss: 0.0015 - val_mae: 0.0317

Epoch 00001: val_mae improved from inf to 0.03175, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 2/1000
7/7 - 0s - loss: 0.0046 - mae: 0.0558 - val_loss: 0.0043 - val_mae: 0.0585

Epoch 00002: val_mae did not improve from 0.03175
Epoch 3/1000
7/7 - 0s - loss: 0.0024 - mae: 0.0411 - val_loss: 0.0028 - val_mae: 0.0449

Epoch 00003: val_mae did not improve from 0.03175
Epoch 4/1000
7/7 - 0s - loss: 0.0014 - mae: 0.0304 - val_loss: 0.0028 - val_mae: 0.0425

Epoch 00004: val_mae did not improve from 0.03175
Epoch 5/1000
7/7 - 0s - loss: 9.2476e-04 - mae: 0.0244 - val_loss: 0.0013 - val_mae: 0.0287

Epoch 00005: val_mae improved from 0.03175 to 0.02866, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 6/1000
7/7 - 0s - loss: 5.7707e-04 - mae: 0.0177 - val_loss: 0.0013 - val_mae: 0.0249

Epoch 00006: val_mae improved from 0.02866 to 0.02489, saving model to ../../saved_mod


Epoch 00055: val_mae did not improve from 0.02432
Epoch 56/1000
7/7 - 0s - loss: 1.0782e-04 - mae: 0.0060 - val_loss: 9.0941e-04 - val_mae: 0.0243

Epoch 00056: val_mae did not improve from 0.02432
Epoch 57/1000
7/7 - 0s - loss: 1.0725e-04 - mae: 0.0060 - val_loss: 9.0679e-04 - val_mae: 0.0243

Epoch 00057: val_mae improved from 0.02432 to 0.02426, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 58/1000
7/7 - 0s - loss: 1.0683e-04 - mae: 0.0060 - val_loss: 9.1428e-04 - val_mae: 0.0244

Epoch 00058: val_mae did not improve from 0.02426
Epoch 59/1000
7/7 - 0s - loss: 1.0612e-04 - mae: 0.0060 - val_loss: 9.3712e-04 - val_mae: 0.0248

Epoch 00059: val_mae did not improve from 0.02426
Epoch 60/1000
7/7 - 0s - loss: 1.0596e-04 - mae: 0.0060 - val_loss: 8.9337e-04 - val_mae: 0.0240

Epoch 00060: val_mae improved from 0.02426 to 0.02403, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 61/1000
7/7 - 0s - loss: 1.0541e-04 - mae: 0.0059 - val_loss: 8.7275e-04 - va


Epoch 00102: val_mae did not improve from 0.02038
Epoch 103/1000
7/7 - 0s - loss: 9.1541e-05 - mae: 0.0055 - val_loss: 6.8546e-04 - val_mae: 0.0205

Epoch 00103: val_mae did not improve from 0.02038
Epoch 104/1000
7/7 - 0s - loss: 9.1176e-05 - mae: 0.0055 - val_loss: 6.8987e-04 - val_mae: 0.0206

Epoch 00104: val_mae did not improve from 0.02038
Epoch 105/1000
7/7 - 0s - loss: 9.1115e-05 - mae: 0.0055 - val_loss: 6.7386e-04 - val_mae: 0.0202

Epoch 00105: val_mae improved from 0.02038 to 0.02024, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 106/1000
7/7 - 0s - loss: 9.0811e-05 - mae: 0.0055 - val_loss: 6.9165e-04 - val_mae: 0.0206

Epoch 00106: val_mae did not improve from 0.02024
Epoch 107/1000
7/7 - 0s - loss: 9.0697e-05 - mae: 0.0055 - val_loss: 6.7426e-04 - val_mae: 0.0203

Epoch 00107: val_mae did not improve from 0.02024
Epoch 108/1000
7/7 - 0s - loss: 9.0476e-05 - mae: 0.0055 - val_loss: 6.6180e-04 - val_mae: 0.0201

Epoch 00108: val_mae improved from 0.02024

7/7 - 0s - loss: 7.7798e-05 - mae: 0.0053 - val_loss: 5.8756e-04 - val_mae: 0.0184

Epoch 00154: val_mae did not improve from 0.01799
Epoch 155/1000
7/7 - 0s - loss: 7.7606e-05 - mae: 0.0053 - val_loss: 5.8764e-04 - val_mae: 0.0184

Epoch 00155: val_mae did not improve from 0.01799
Epoch 156/1000
7/7 - 0s - loss: 7.7598e-05 - mae: 0.0053 - val_loss: 5.9435e-04 - val_mae: 0.0184

Epoch 00156: val_mae did not improve from 0.01799
Epoch 157/1000
7/7 - 0s - loss: 7.7488e-05 - mae: 0.0053 - val_loss: 5.5969e-04 - val_mae: 0.0178

Epoch 00157: val_mae improved from 0.01799 to 0.01778, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 158/1000
7/7 - 0s - loss: 7.7256e-05 - mae: 0.0053 - val_loss: 5.5163e-04 - val_mae: 0.0177

Epoch 00158: val_mae improved from 0.01778 to 0.01768, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 159/1000
7/7 - 0s - loss: 7.6531e-05 - mae: 0.0053 - val_loss: 5.6915e-04 - val_mae: 0.0180

Epoch 00159: val_mae did not improve from 0.0

7/7 - 0s - loss: 7.0203e-05 - mae: 0.0051 - val_loss: 5.5430e-04 - val_mae: 0.0176

Epoch 00206: val_mae did not improve from 0.01684
Epoch 207/1000
7/7 - 0s - loss: 7.0329e-05 - mae: 0.0051 - val_loss: 5.5308e-04 - val_mae: 0.0174

Epoch 00207: val_mae did not improve from 0.01684
Epoch 208/1000
7/7 - 0s - loss: 7.0361e-05 - mae: 0.0051 - val_loss: 5.2932e-04 - val_mae: 0.0171

Epoch 00208: val_mae did not improve from 0.01684
Epoch 209/1000
7/7 - 0s - loss: 7.0233e-05 - mae: 0.0051 - val_loss: 5.4804e-04 - val_mae: 0.0175

Epoch 00209: val_mae did not improve from 0.01684
Epoch 210/1000
7/7 - 0s - loss: 7.0244e-05 - mae: 0.0051 - val_loss: 5.4537e-04 - val_mae: 0.0173

Epoch 00210: val_mae did not improve from 0.01684
Epoch 211/1000
7/7 - 0s - loss: 7.0164e-05 - mae: 0.0051 - val_loss: 5.1328e-04 - val_mae: 0.0168

Epoch 00211: val_mae improved from 0.01684 to 0.01675, saving model to ../../saved_models/water_all_95_ft_cp1.h5
Epoch 212/1000
7/7 - 0s - loss: 7.0982e-05 - mae: 0.0052 -


Epoch 00258: val_mae did not improve from 0.01606
Epoch 259/1000
7/7 - 0s - loss: 7.0026e-05 - mae: 0.0052 - val_loss: 5.9361e-04 - val_mae: 0.0182

Epoch 00259: val_mae did not improve from 0.01606
Epoch 260/1000
7/7 - 0s - loss: 6.9358e-05 - mae: 0.0051 - val_loss: 5.4678e-04 - val_mae: 0.0172

Epoch 00260: val_mae did not improve from 0.01606
Epoch 261/1000
7/7 - 0s - loss: 6.8311e-05 - mae: 0.0051 - val_loss: 5.4187e-04 - val_mae: 0.0172

Epoch 00261: val_mae did not improve from 0.01606
Epoch 262/1000
7/7 - 0s - loss: 6.7985e-05 - mae: 0.0050 - val_loss: 5.6505e-04 - val_mae: 0.0177

Epoch 00262: val_mae did not improve from 0.01606
Epoch 263/1000
7/7 - 0s - loss: 6.8062e-05 - mae: 0.0051 - val_loss: 5.7548e-04 - val_mae: 0.0178

Epoch 00263: val_mae did not improve from 0.01606
Epoch 264/1000
7/7 - 0s - loss: 6.8422e-05 - mae: 0.0051 - val_loss: 5.7654e-04 - val_mae: 0.0178

Epoch 00264: val_mae did not improve from 0.01606
Epoch 265/1000
7/7 - 0s - loss: 6.8349e-05 - mae: 0.005


Epoch 00313: val_mae did not improve from 0.01567
Epoch 314/1000
7/7 - 0s - loss: 7.1532e-05 - mae: 0.0054 - val_loss: 6.5273e-04 - val_mae: 0.0195

Epoch 00314: val_mae did not improve from 0.01567
Epoch 315/1000
7/7 - 0s - loss: 7.9913e-05 - mae: 0.0059 - val_loss: 5.6708e-04 - val_mae: 0.0177

Epoch 00315: val_mae did not improve from 0.01567
Epoch 316/1000
7/7 - 0s - loss: 7.3820e-05 - mae: 0.0055 - val_loss: 5.0475e-04 - val_mae: 0.0164

Epoch 00316: val_mae did not improve from 0.01567
Epoch 317/1000
7/7 - 0s - loss: 7.0333e-05 - mae: 0.0053 - val_loss: 5.1309e-04 - val_mae: 0.0165

Epoch 00317: val_mae did not improve from 0.01567
Epoch 318/1000
7/7 - 0s - loss: 7.4135e-05 - mae: 0.0056 - val_loss: 5.7700e-04 - val_mae: 0.0180

Epoch 00318: val_mae did not improve from 0.01567
Epoch 319/1000
7/7 - 0s - loss: 6.7640e-05 - mae: 0.0051 - val_loss: 5.7258e-04 - val_mae: 0.0178

Epoch 00319: val_mae did not improve from 0.01567
Epoch 320/1000
7/7 - 0s - loss: 7.1032e-05 - mae: 0.005


Epoch 00368: val_mae did not improve from 0.01565
Epoch 369/1000
7/7 - 0s - loss: 8.1839e-05 - mae: 0.0061 - val_loss: 6.2427e-04 - val_mae: 0.0189

Epoch 00369: val_mae did not improve from 0.01565
Epoch 370/1000
7/7 - 0s - loss: 8.3841e-05 - mae: 0.0063 - val_loss: 5.6062e-04 - val_mae: 0.0175

Epoch 00370: val_mae did not improve from 0.01565
Epoch 371/1000
7/7 - 0s - loss: 6.9149e-05 - mae: 0.0052 - val_loss: 5.7770e-04 - val_mae: 0.0178

Epoch 00371: val_mae did not improve from 0.01565
Epoch 372/1000
7/7 - 0s - loss: 6.9218e-05 - mae: 0.0053 - val_loss: 5.9202e-04 - val_mae: 0.0180

Epoch 00372: val_mae did not improve from 0.01565
Epoch 373/1000
7/7 - 0s - loss: 7.5494e-05 - mae: 0.0057 - val_loss: 4.9087e-04 - val_mae: 0.0160

Epoch 00373: val_mae did not improve from 0.01565
Epoch 374/1000
7/7 - 0s - loss: 9.6631e-05 - mae: 0.0070 - val_loss: 6.0872e-04 - val_mae: 0.0186

Epoch 00374: val_mae did not improve from 0.01565
Epoch 375/1000
7/7 - 0s - loss: 7.4211e-05 - mae: 0.005


Epoch 00423: val_mae did not improve from 0.01548
Epoch 424/1000
7/7 - 0s - loss: 7.0042e-05 - mae: 0.0053 - val_loss: 5.9567e-04 - val_mae: 0.0183

Epoch 00424: val_mae did not improve from 0.01548
Epoch 425/1000
7/7 - 0s - loss: 6.5761e-05 - mae: 0.0051 - val_loss: 5.3411e-04 - val_mae: 0.0171

Epoch 00425: val_mae did not improve from 0.01548
Epoch 426/1000
7/7 - 0s - loss: 7.1091e-05 - mae: 0.0054 - val_loss: 7.7034e-04 - val_mae: 0.0215

Epoch 00426: val_mae did not improve from 0.01548
Epoch 427/1000
7/7 - 0s - loss: 1.2639e-04 - mae: 0.0086 - val_loss: 4.8048e-04 - val_mae: 0.0157

Epoch 00427: val_mae did not improve from 0.01548
Epoch 428/1000
7/7 - 0s - loss: 9.1262e-05 - mae: 0.0067 - val_loss: 4.7258e-04 - val_mae: 0.0158

Epoch 00428: val_mae did not improve from 0.01548
Epoch 429/1000
7/7 - 0s - loss: 8.0093e-05 - mae: 0.0060 - val_loss: 5.3045e-04 - val_mae: 0.0171

Epoch 00429: val_mae did not improve from 0.01548
Epoch 430/1000
7/7 - 0s - loss: 6.8175e-05 - mae: 0.005


Epoch 00478: val_mae did not improve from 0.01548
Epoch 479/1000
7/7 - 0s - loss: 7.5591e-05 - mae: 0.0058 - val_loss: 6.4942e-04 - val_mae: 0.0195

Epoch 00479: val_mae did not improve from 0.01548
Epoch 480/1000
7/7 - 0s - loss: 6.8673e-05 - mae: 0.0053 - val_loss: 6.3644e-04 - val_mae: 0.0190

Epoch 00480: val_mae did not improve from 0.01548
Epoch 481/1000
7/7 - 0s - loss: 6.5639e-05 - mae: 0.0051 - val_loss: 6.5173e-04 - val_mae: 0.0195

Epoch 00481: val_mae did not improve from 0.01548
Epoch 482/1000
7/7 - 0s - loss: 6.7858e-05 - mae: 0.0053 - val_loss: 5.6471e-04 - val_mae: 0.0174

Epoch 00482: val_mae did not improve from 0.01548
Epoch 483/1000
7/7 - 0s - loss: 6.5714e-05 - mae: 0.0051 - val_loss: 6.1530e-04 - val_mae: 0.0186

Epoch 00483: val_mae did not improve from 0.01548
Epoch 484/1000
7/7 - 0s - loss: 7.0750e-05 - mae: 0.0054 - val_loss: 4.8198e-04 - val_mae: 0.0158

Epoch 00484: val_mae did not improve from 0.01548
Epoch 485/1000
7/7 - 0s - loss: 1.3214e-04 - mae: 0.008